In [16]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Third Attempt, drop two more columns, STATUS and SPECIAL_CONSIDERATIONS to check if accuracy improves

In [17]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [18]:
# Determine which values to replace if counts are less than ...?
application_counts = application_df.APPLICATION_TYPE.value_counts()
replace_application = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [19]:
# Determine which values to replace if counts are less than ..?
classification_counts = application_df.CLASSIFICATION.value_counts()
replace_class = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [20]:
# Generate our categorical variable list
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [21]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

/Users/nikki/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

/Users/nikki/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [24]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3280      
                                                                 
 dense_4 (Dense)             (None, 50)                4050      
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 7,381
Trainable params: 7,381
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5695 - accuracy: 0.7225
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5562 - accuracy: 0.7292
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5529 - accuracy: 0.7303
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5512 - accuracy: 0.7315
Epoch 5/100
791/804 [============================>.] - ETA: 0s - loss: 0.5510 - accuracy: 0.7316
Epoch 00005: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5504 - accuracy: 0.7322
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5490 - accuracy: 0.7335
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7323
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5472 - accuracy: 0.7333
Epoch 9/100
804/804 [=============================

804/804 [==============================] - 1s 1ms/step - loss: 0.5376 - accuracy: 0.7403
Epoch 64/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5369 - accuracy: 0.7402
Epoch 65/100
772/804 [===========================>..] - ETA: 0s - loss: 0.5371 - accuracy: 0.7404
Epoch 00065: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5373 - accuracy: 0.7409
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5369 - accuracy: 0.7407
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5370 - accuracy: 0.7403
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5368 - accuracy: 0.7406
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5371 - accuracy: 0.7399
Epoch 70/100
778/804 [============================>.] - ETA: 0s - loss: 0.5381 - accuracy: 0.7403
Epoch 00070: saving model to checkpoints/weights.70.hdf5
80

In [29]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

### Second Attempt - Using tahn activation in the first layer

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3280      
                                                                 
 dense_7 (Dense)             (None, 50)                4050      
                                                                 
 dense_8 (Dense)             (None, 1)                 51        
                                                                 
Total params: 7,381
Trainable params: 7,381
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [33]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5655 - accuracy: 0.7242
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5556 - accuracy: 0.7304
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5528 - accuracy: 0.7310
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5519 - accuracy: 0.7324
Epoch 5/100
784/804 [============================>.] - ETA: 0s - loss: 0.5497 - accuracy: 0.7325
Epoch 00005: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5497 - accuracy: 0.7327
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5501 - accuracy: 0.7320: 0s - loss: 0.5515 - accuracy
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5485 - accuracy: 0.7333
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5475 - accuracy: 0.7342
Epoch 9/100
804/804 

804/804 [==============================] - 1s 1ms/step - loss: 0.5368 - accuracy: 0.7404
Epoch 63/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5367 - accuracy: 0.7406
Epoch 64/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5366 - accuracy: 0.7412
Epoch 65/100
789/804 [============================>.] - ETA: 0s - loss: 0.5366 - accuracy: 0.7409
Epoch 00065: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5369 - accuracy: 0.7409
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5368 - accuracy: 0.7405
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5364 - accuracy: 0.7402
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5359 - accuracy: 0.7405
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5368 - accuracy: 0.7409
Epoch 70/100
772/804 [===========================>..] -

### Fourth Attempt - adding another hidden layer

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 80)                3280      
                                                                 
 dense_10 (Dense)            (None, 50)                4050      
                                                                 
 dense_11 (Dense)            (None, 30)                1530      
                                                                 
 dense_12 (Dense)            (None, 1)                 31        
                                                                 
Total params: 8,891
Trainable params: 8,891
Non-trainable params: 0
_________________________________________________________________


In [36]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [37]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [38]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5700 - accuracy: 0.7228
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5543 - accuracy: 0.7306
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5520 - accuracy: 0.7305
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5503 - accuracy: 0.7333
Epoch 5/100
798/804 [============================>.] - ETA: 0s - loss: 0.5496 - accuracy: 0.7327
Epoch 00005: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5494 - accuracy: 0.7327
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5486 - accuracy: 0.7345
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5477 - accuracy: 0.7337
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5476 - accuracy: 0.7350
Epoch 9/100
804/804 [=============================

Epoch 63/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5368 - accuracy: 0.7407
Epoch 64/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5365 - accuracy: 0.7404
Epoch 65/100
798/804 [============================>.] - ETA: 0s - loss: 0.5369 - accuracy: 0.7400
Epoch 00065: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5367 - accuracy: 0.7401
Epoch 66/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5362 - accuracy: 0.7411: 0s - l
Epoch 67/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5365 - accuracy: 0.7402
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5362 - accuracy: 0.7405
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5366 - accuracy: 0.7403: 0s - los
Epoch 70/100
769/804 [===========================>..] - ETA: 0s - loss: 0.5363 - accuracy: 0.7407
Epoch 00070: saving model to

### Fifth Attempt - without callback

In [39]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 2ms/step - loss: 22061.2051 - accuracy: 0.5067
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 5996.4741 - accuracy: 0.5015
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 1692.2930 - accuracy: 0.5023
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 1082.7670 - accuracy: 0.5108 0s - loss: 163.0920 - accuracy
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 99.7480 - accuracy: 0.5199
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6988 - accuracy: 0.5321
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6924 - accuracy: 0.5321
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 10/100
804/804 [========================

804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 80/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 81/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 82/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 85/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 86/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6911 - accuracy: 0.5321: 0s - loss: 0.6910 - accu
Epoch 88/100
804/804 [==============================] - 2s 2ms/step -

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6909 - accuracy: 0.5332 - 265ms/epoch - 987us/step
Loss: 0.6909478306770325, Accuracy: 0.5331778526306152
